# Logistic Regression Voting Model

## 00. TODO

1. Generate predictions for all
2. Voting Classifier

## 0. Imports

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn as sk
assert sk.__version__ >= "0.20"

# XGBoost Import
import xgboost as xgb
xgb.set_config(verbosity = 2)
# assert xgb.get_config["verbosity"] == 0

# Common imports
import numpy as np
import os
import pandas as pd
import seaborn as sns

# to make this notebook's output stable across runs
np.random.seed(0)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from IPython.display import display, HTML

pd.set_option('display.max_columns', None)

# Get current directory of file (so filesystem should work regardless of user machine)
print(os.getcwd())
parent = os.pardir
print(parent)

# Read in data from sources
play_by_play_path = os.path.join(parent, 'Data')
play_by_play_path = os.path.join(play_by_play_path, 'syracuse_gsw_basic_pbp.csv')
play_by_play_df = pd.read_csv(play_by_play_path)

results_path = os.path.join(parent, 'Data')
results_path = os.path.join(results_path, "syracuse_gsw_basic_results.csv")
game_results_df = pd.read_csv(results_path)

game_data_master_path = os.path.join(parent, 'Data')
game_data_master_path = os.path.join(game_data_master_path, 'game_data_master.csv')
game_data_master = pd.read_csv(game_data_master_path)



# Reset indices
play_by_play_df.set_index('id', inplace= True)
game_results_df.set_index('id', inplace= True)
display(play_by_play_df.head(2))


import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

C:\Users\perso\anaconda3\envs\Shark_env\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


C:\Users\perso\Documents\GitHub\syracuseBasketballCompetition\Player Models
..


,season,season_type,nba_game_id,home,away,game_date,quarter,poss_id,off_team,def_team,poss_time,oncourt_id,play_type,off_def,player_type,player_name,player_nba_id,play_clock,sequence_id,play_id,result_id,seq_result_id,play_zone,last_updated_time
id,,,,,,,,,,,,,,,,,,,,,,,,
1219,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,1,LAL,GSW,720,1,Rim Gather,OFF,Ball-Handler,DeAndre Jordan,201599.0,716,1,2,0022100002_1_1,0022100002_1_1,Rim,26:31.2
1227,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,3,LAL,GSW,694,1,Initiation,OFF,Ball-Handler,Russell Westbrook,201566.0,689,3,6,NaN,0022100002_1_3,Mid-Left Above Break 3,26:31.2


## 1. Model & CV

In [2]:
game_data_master['points'].fillna(0, inplace= True)


In [3]:
game_data_master = game_data_master.loc[(game_data_master['season_type'] == 'Regular Season')]


In [4]:
game_data_master = game_data_master.loc[(game_data_master['player_name'] == 'Stephen Curry')]


In [5]:
game_data_master = game_data_master.loc[
                                         (game_data_master['result_type'] == 'Face-Up') |
                                         (game_data_master['result_type'] == 'Layup') |
                                         (game_data_master['result_type'] == 'Floater') |
                                         (game_data_master['result_type'] == '1-2D Pull-Up') |
                                         (game_data_master['result_type'] == 'Off-Movement') |
                                         (game_data_master['result_type'] == 'Catch & Shoot') |
                                         (game_data_master['result_type'] == 'Off-Lob / Tip') |
                                         (game_data_master['result_type'] == '3+D Pull-Up') |
                                         (game_data_master['result_type'] == 'Take Side') |
                                         (game_data_master['result_type'] == 'Post Fade') |
                                         (game_data_master['result_type'] == 'Hook') |
                                         #(game_data_master['result_type'] == 'Putback') |
                                         (game_data_master['result_type'] == 'Leaner') |
                                         (game_data_master['result_type'] == 'PU Fade') |
                                         (game_data_master['result_type'] == 'Dunk')
                                       ]


In [6]:

group = game_data_master.groupby('result_type')
game_data_master = group.filter(lambda x: len(x) > 10) 
display(game_data_master['result_type'].value_counts())


group = game_data_master.groupby('result_contest')
game_data_master = group.filter(lambda x: len(x) > 10) 
game_data_master = game_data_master.loc[ (game_data_master['result_contest'] != 'Fouled') &
                                         (game_data_master['result_contest'] != 'Block') &
                                         (game_data_master['result_contest'] != 'Alter') ]

display(game_data_master['result_contest'].value_counts())

group = game_data_master.groupby('result_zone')
game_data_master = group.filter(lambda x: len(x) > 10) 
display(game_data_master['result_zone'].value_counts())


group = game_data_master.groupby('points')
game_data_master = group.filter(lambda x: len(x) > 10) 
display(game_data_master['points'].value_counts())





Layup            662
Catch & Shoot    596
Off-Movement     434
3+D Pull-Up      392
Floater          256
1-2D Pull-Up     222
Dunk             122
Take Side         74
PU Fade           66
Face-Up           50
Hook              29
Off-Lob / Tip     21
Leaner            19
Post Fade         15
Name: result_type, dtype: int64

Average    1185
Plus        644
Open        506
Poor        189
Name: result_contest, dtype: int64

Rim                        715
Right Above Break 3        318
Mid-Left Above Break 3     288
Left Above Break 3         265
Mid-Right Above Break 3    250
Right Corner 3             203
Left Corner 3              187
Left Short Mid              48
Mid-Right Midrange          48
Mid-Left Midrange           40
Right Midrange              39
Left Midrange               36
Right Short Mid             27
Mid-Left Short Paint        21
Mid-Right Short Paint       19
Mid-Left Deep Paint         11
Name: result_zone, dtype: int64

0.0    1293
2.0     640
3.0     582
Name: points, dtype: int64

In [7]:
targets = ['points']

attributes = [  
                'player_name',
                'result_type',
                'result_contest',
                'result_zone',
            ]




In [8]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(game_data_master[attributes])
y = game_data_master[targets]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=0)


In [9]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, auc

C_grid = [1E-5,1E-4,1E-3,1E-2,1E-1,1E0]


In [10]:

# group / ensemble of models
estimator = []
estimator.append(('LR1', 
                  LogisticRegressionCV(solver ='lbfgs', 
                                     multi_class ='ovr',
                                     Cs = C_grid, 
                                     random_state=69,
                                     max_iter = 10000)))
estimator.append(('LR2', 
                  LogisticRegressionCV(solver ='lbfgs', 
                                     multi_class ='auto',
                                     Cs = C_grid, 
                                     random_state=420, 
                                     max_iter = 10000)))
estimator.append(('LR3', 
                  LogisticRegressionCV(solver ='lbfgs', 
                                     multi_class ='ovr',
                                     Cs = C_grid, 
                                     random_state=0,
                                     max_iter = 10000)))
estimator.append(('LR4', 
                  LogisticRegressionCV(solver ='lbfgs', 
                                     multi_class ='multinomial', 
                                     Cs = C_grid, 
                                     random_state=9876756,
                                     max_iter = 10000)))
estimator.append(('LR5', 
                  LogisticRegressionCV(solver ='lbfgs', 
                                     multi_class ='multinomial',
                                     Cs = C_grid, 
                                     random_state=345,
                                     max_iter = 10000)))
  

In [11]:
# Voting Classifier with hard voting
vot_soft = VotingClassifier(estimators = estimator, voting ='soft')
vot_soft.fit(X_train, y_train)



VotingClassifier(estimators=[('LR1',
                              LogisticRegressionCV(Cs=[1e-05, 0.0001, 0.001,
                                                       0.01, 0.1, 1.0],
                                                   max_iter=10000,
                                                   multi_class='ovr',
                                                   random_state=69)),
                             ('LR2',
                              LogisticRegressionCV(Cs=[1e-05, 0.0001, 0.001,
                                                       0.01, 0.1, 1.0],
                                                   max_iter=10000,
                                                   random_state=420)),
                             ('LR3',
                              LogisticRegressionCV(Cs=[1e-05, 0.0001, 0.001,
                                                       0.01, 0.1, 1.0],
                                                   max_iter=10000,
                                                   multi_class='ovr',
                                                   random_state=0)),
                             ('LR4',
                              LogisticRegressionCV(Cs=[1e-05, 0.0001, 0.001,
                                                       0.01, 0.1, 1.0],
                                                   max_iter=10000,
                                                   multi_class='multinomial',
                                                   random_state=9876756)),
                             ('LR5',
                              LogisticRegressionCV(Cs=[1e-05, 0.0001, 0.001,
                                                       0.01, 0.1, 1.0],
                                                   max_iter=10000,
                                                   multi_class='multinomial',
                                                   random_state=345))],
                 voting='soft')

In [12]:
_accuracy_score = np.average(cross_val_score(vot_soft, X_train, y_train, cv=4, scoring="accuracy")) # Avg Cross val score
_f1_score = np.average(cross_val_score(vot_soft, X_train, y_train, cv=4, scoring="f1_weighted")) # Avg Cross val score

In [13]:
_accuracy_score, _f1_score #, _roc_auc_score, , _auc

(0.6426444082317939, 0.557857090604446)

In [14]:
log_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", penalty="l2", C = 1.0, random_state=0, max_iter=10000)

np.average(cross_val_score(log_reg, X_train, y_train, cv=4, scoring="accuracy")) # Avg Cross val score

#f1_weighted = weighted f1 by class instances

0.638911521795489

## 2. Predictions

In order to get the correct number of columns for the one hot encoder (pd.get_dummies()), we must add it to the end of the 
game data then take the one hot. This ensures that there are the correct number of columns from the one hot. 

### Scenarios


In [15]:
game_data_master['result_type'].unique()

array(['Floater', 'Face-Up', 'Off-Movement', 'Catch & Shoot', 'Layup',
       '3+D Pull-Up', '1-2D Pull-Up', 'Off-Lob / Tip', 'Hook', 'PU Fade',
       'Leaner', 'Dunk', 'Post Fade'], dtype=object)

In [16]:

predict_series = []

for player in  game_data_master['player_name'].unique():
    for result in game_data_master['result_type'].unique():
        for contest in game_data_master['result_contest'].unique():
            for zone in game_data_master['result_zone'].unique():
                predict_series.append({
                    'player_name': player,
                    'result_type': result,
                    'result_contest': contest,
                    'result_zone': zone
                       })

predict_series = pd.DataFrame(predict_series)

In [17]:
predict_series.head()

,player_name,result_type,result_contest,result_zone
0,Stephen Curry,Floater,Average,Rim
1,Stephen Curry,Floater,Average,Right Corner 3
2,Stephen Curry,Floater,Average,Mid-Left Above Break 3
3,Stephen Curry,Floater,Average,Left Corner 3
4,Stephen Curry,Floater,Average,Mid-Right Above Break 3


In [18]:
predict_probs = pd.DataFrame()

for i in range(len(predict_series)):

    scenario = predict_series.iloc[i]
    scenario = pd.DataFrame(scenario).transpose()

    # New DF
    game_data_master_predict = game_data_master    


    ## Append to end of master df
    game_data_master_predict = pd.concat([game_data_master_predict, scenario]) 



    ## Filter out the unimportant columns
    game_data_master_predict = game_data_master_predict[attributes]


    ## one hot encode
    game_data_master_predict = pd.get_dummies(game_data_master_predict)



    ## Get the last entity (the predict_series)
    predicts = game_data_master_predict.iloc[-1]

    ## Turn it into a DataFrame
    predicts = pd.DataFrame(predicts) 

    ## Transpose so that it is 2d array
    predicts = predicts.transpose() 

    probs = pd.DataFrame(vot_soft.predict_proba(predicts))


    output = scenario.to_dict(orient='records')[0] | probs.to_dict(orient='records')[0]

    predict_probs = predict_probs.append(output, ignore_index= True)


In [19]:
predict_probs.rename(columns= {0: '0 pts', 
                      1: '2 pts',
                      2: '3 pts'
                     }, inplace= True)
predict_probs

,player_name,result_type,result_contest,result_zone,0 pts,2 pts,3 pts
0,Stephen Curry,Floater,Average,Rim,0.397001,0.455788,0.147211
1,Stephen Curry,Floater,Average,Right Corner 3,0.603516,0.162927,0.233557
2,Stephen Curry,Floater,Average,Mid-Left Above Break 3,0.588951,0.147980,0.263069
3,Stephen Curry,Floater,Average,Left Corner 3,0.597107,0.170159,0.232733
4,Stephen Curry,Floater,Average,Mid-Right Above Break 3,0.620989,0.149014,0.229997
...,...,...,...,...,...,...,...
827,Stephen Curry,Post Fade,Poor,Mid-Left Short Paint,0.542763,0.237121,0.220116
828,Stephen Curry,Post Fade,Poor,Mid-Left Midrange,0.522616,0.265829,0.211555
829,Stephen Curry,Post Fade,Poor,Mid-Left Deep Paint,0.536664,0.241800,0.221536
830,Stephen Curry,Post Fade,Poor,Left Midrange,0.531646,0.256558,0.211795


In [21]:
%pwd

'C:\\Users\\perso\\Documents\\GitHub\\syracuseBasketballCompetition\\Player Models'

In [20]:
#%cd ../
predict_probs.to_csv(r'./Curry_data/curry_regszn.csv')